In [10]:
import scrapetube
import json

videos = list(scrapetube.get_channel("UCWsDFcIhY2DBi3GB5uykGXA", content_type="streams"))
with open("videos.json", "w", encoding="utf-8") as f:
    json.dump(videos, f, ensure_ascii=False, indent=2)


In [18]:
import pandas as pd
import json
import regex as re
import scrapetube
from youtube_comment_downloader import *
from itertools import islice

In [ ]:


with open("videos.json", "r", encoding="utf-8") as f:
    data = json.load(f)

df = pd.DataFrame([
    {
        "videoId": v.get("videoId"),
        "title": v.get("title", {}).get("runs", [{}])[0].get("text"),
        "published": v.get("publishedTimeText", {}).get("simpleText"),
        "duration": v.get("lengthText", {}).get("simpleText")
    }
    for v in data
])

df

,videoId,title,published,duration
0,vdQNKVplOCg,LOVING STREAMER + WORKING OUT💪🏽 + TEACHING JA...,Streamed 1 day ago,3:25:29
1,4WOrNwG8Uu8,PLAYING LAST OF US (PART 1 and 2) UNTIL I BEAT...,Streamed 4 days ago,7:13:45
2,dFBXskxYWOQ,PLAYING LAST OF US (PART 1 and 2) UNTIL I BEAT...,Streamed 4 days ago,11:51:56
3,5t4vF5vuVuM,PLAYING LAST OF US (PART 1 and 2) UNTIL I BEAT IT,Streamed 5 days ago,11:54:56
4,mdTHZN89I5k,ITS OVER + SPIN THE WHEEL FOR $$$,Streamed 9 days ago,4:56:53
...,...,...,...,...
1085,0SIMB8etbYo,Keyboard and mouse player (Ps4) Solo Dubs,Streamed 6 years ago,28:54
1086,SBUlJ04YPm8,Keyboard and mouse player (Ps4) Solo Dubs,Streamed 6 years ago,1:14:24
1087,lwwg1HMQx4s,Keyboard and mouse player (Ps4) Solo Dubs,Streamed 6 years ago,1:06:25
1088,lO7dgiOgJ84,Keyboard and mouse player (Ps4) Solo Dubs,Streamed 6 years ago,1:11:48


In [ ]:
from datetime import datetime, timedelta
import re

def parse_relative_date(text, ref=None):
    if ref is None:
        ref = datetime.today()
    text = text.lower().strip()

    match = re.search(r"(\d+)\s+(minute|hour|day|week|month|year)", text)
    if not match:
        return ref.date().isoformat()

    n, unit = int(match.group(1)), match.group(2)
    if unit == "minute":
        delta = timedelta(minutes=n)
    elif unit == "hour":
        delta = timedelta(hours=n)
    elif unit == "day":
        delta = timedelta(days=n)
    elif unit == "week":
        delta = timedelta(weeks=n)
    elif unit == "month":
        delta = timedelta(days=30 * n)
    elif unit == "year":
        delta = timedelta(days=365 * n)

    return (ref - delta).date().isoformat()

df["published_date"] = df["published"].apply(parse_relative_date)

In [16]:
df["published_date"] = pd.to_datetime(df["published_date"])
dk = df[df['published_date'] >= "2023-01-01"]

In [17]:
dk

,videoId,title,published,duration,published_date
0,vdQNKVplOCg,LOVING STREAMER + WORKING OUT💪🏽 + TEACHING JA...,Streamed 1 day ago,3:25:29,2025-05-21
1,4WOrNwG8Uu8,PLAYING LAST OF US (PART 1 and 2) UNTIL I BEAT...,Streamed 4 days ago,7:13:45,2025-05-18
2,dFBXskxYWOQ,PLAYING LAST OF US (PART 1 and 2) UNTIL I BEAT...,Streamed 4 days ago,11:51:56,2025-05-18
3,5t4vF5vuVuM,PLAYING LAST OF US (PART 1 and 2) UNTIL I BEAT IT,Streamed 5 days ago,11:54:56,2025-05-17
4,mdTHZN89I5k,ITS OVER + SPIN THE WHEEL FOR $$$,Streamed 9 days ago,4:56:53,2025-05-13
...,...,...,...,...,...
538,u76ytxKniBo,bad hair day,Streamed 2 years ago,1:05:42,2023-05-23
539,OT7yAMjVyqI,8million......,Streamed 2 years ago,1:26:12,2023-05-23
540,b5isjIS8veE,we cool?,Streamed 2 years ago,3:21:35,2023-05-23
541,93Jx5a9uhdQ,HAAAAAAA,Streamed 2 years ago,4:06:57,2023-05-23


In [19]:
def get_url(video_id):
    url = "https://www.youtube.com/watch?v=" + video_id
    return url 

def get_comments(video_id):
    youtube_url = get_url(video_id)
    comments = downloader.get_comments_from_url(youtube_url)
    comments_list = list(comments)
    with open(f'{video_id}.json', 'w') as json_file:
        json.dump(comments_list, json_file, indent=4)
    return f'{video_id}.json created'

In [21]:
downloader = YoutubeCommentDownloader()
id_list = [m for m in dk['videoId']]
[get_comments(m) for m in id_list]

['vdQNKVplOCg.json created',
 '4WOrNwG8Uu8.json created',
 'dFBXskxYWOQ.json created',
 '5t4vF5vuVuM.json created',
 'mdTHZN89I5k.json created',
 'tdn4vTUDIeo.json created',
 'fam0AWzv0g0.json created',
 'Bz0hb76kdwc.json created',
 'N32DiyCPzKo.json created',
 'bjVWn28I07M.json created',
 'r9UA3ondtaw.json created',
 'YdZntExmo_k.json created',
 'o3N-9nklu3U.json created',
 '0mwuGExoxFU.json created',
 '7cAzQjnEqXs.json created',
 'zTW20UFOVbo.json created',
 'oNS8PHxWdp8.json created',
 'QtIlD6uxwwk.json created',
 'g4TDjwPRArg.json created',
 'zBL_5DkiXCk.json created',
 'K8d4n9GhBrg.json created',
 'fK85SQzm0Z0.json created',
 'oTx6EXTQNSQ.json created',
 'rX7kyY2lWTM.json created',
 'SahEdZYx7Ds.json created',
 '-e3j2xzAakg.json created',
 '5BixN1cdMwg.json created',
 'IOknw-sk3ns.json created',
 'EE90rnGMBGI.json created',
 'xFe7M0bFXSo.json created',
 'so1Y06pQ_UM.json created',
 'p36Nhm-PZKc.json created',
 'OJq1w0DVheE.json created',
 'O3Sp6OqKGxA.json created',
 'gDrGbdEPcdw.